In [1]:
import requests
from requests.auth import HTTPBasicAuth

In [2]:
URL = "https://suite-endpoint-api-apne2.superb-ai.com/endpoints/3b38ff5e-cd49-4d00-a268-0f00df940520/inference"
ACCESS_KEY = "sVuknNs5mQ3PeSkNkMtvx1mmCTJQsqGj5JqhzSb6"

In [3]:
IMAGE_FILE_PATH = "/home/jeonghan/image_box/256.jpg" #90,110,158,[152,164,166,168]-빛?,276-초점/ 96구멍/ 134구멍에 붓셀??,136,192-붓셀2개?
#290- ㅇㅅㅂ인식이미지잘려서

#전체적으로 홀의 인식률이 많이 떨어짐 0% 
# 빛번짐으로 인한초점이안맞는 경우 - 152,164,166,168


#298-5/290-4 유에스비짤림/276-1칩셋하나인식/

#정확도를 높이기위한해결방법
#유에스비 영역조금 더작게-> 구멍인식률을 높이기??
#

In [4]:
image = open(IMAGE_FILE_PATH, "rb").read() # Python에서 이미지를 바이너리 모드로 열고 읽어들임

In [5]:
response = requests.post( #requests 라이브러리를 사용하여 HTTP POST 요청/ 이미지를 서버에 전송
    url=URL,
    auth=HTTPBasicAuth("kdt2024_1-5", ACCESS_KEY), #팀정보, 엑세스키
    headers={"Content-Type": "image/jpeg"},
    data=image,
)

In [6]:
print(response.json()) #서버의 응답 본문을 JSON 형식으로 파싱하여 Python 객체로 변환

# {'objects': [{'class': 'RASPBERRY PICO', 'score': 0.822075605392456, 
# 'box': [173, 280, 436, 606]}, {'class': 'HOLE', 'score': 0.4158550500869751, 
# 'box': [251, 295, 264, 309]}, {'class': 'USB', 'score': 0.39888837933540344, 
# 'box': [357, 533, 416, 593]}, {'class': 'HOLE', 'score': 0.2648520767688751, 
# 'box': [246, 295, 260, 313]}, {'class': 'CHIPSET', 'score': 0.2622401714324951, 
# 'box': [270, 403, 334, 463]}]}

response.status_code  # 4 is bad


{'objects': [{'class': 'RASPBERRY PICO', 'score': 0.8948777318000793, 'box': [156, 259, 375, 591]}, {'class': 'USB', 'score': 0.7243542075157166, 'box': [285, 529, 342, 584]}, {'class': 'CHIPSET', 'score': 0.5630953907966614, 'box': [234, 385, 291, 438]}, {'class': 'OSCILLATOR', 'score': 0.4605947732925415, 'box': [249, 350, 277, 370]}, {'class': 'BOOTSEL', 'score': 0.3982420265674591, 'box': [299, 471, 327, 504]}, {'class': 'HOLE', 'score': 0.2898400127887726, 'box': [248, 266, 258, 277]}]}


200

In [7]:
import cv2
# response.json() -> class/start/end

# 변환된 결과를 저장할 리스트
transformed_objects = []
data = response.json()
# 데이터 변환
for obj in data["objects"]:
    transformed_obj = {
        "class": obj["class"],
        "start_point": [obj["box"][0], obj["box"][1]], #box 첫번째 두번째값
        "end_point": [obj["box"][2], obj["box"][3]] #box 두번째 세번째값을 
    }
    transformed_objects.append(transformed_obj)

# 변환 결과 출력
print(transformed_objects)


[{'class': 'RASPBERRY PICO', 'start_point': [156, 259], 'end_point': [375, 591]}, {'class': 'USB', 'start_point': [285, 529], 'end_point': [342, 584]}, {'class': 'CHIPSET', 'start_point': [234, 385], 'end_point': [291, 438]}, {'class': 'OSCILLATOR', 'start_point': [249, 350], 'end_point': [277, 370]}, {'class': 'BOOTSEL', 'start_point': [299, 471], 'end_point': [327, 504]}, {'class': 'HOLE', 'start_point': [248, 266], 'end_point': [258, 277]}]


In [8]:
import cv2

# 이미지 파일 경로 설정
img = cv2.imread(IMAGE_FILE_PATH)

# 이미지가 제대로 로드되었는지 확인
if img is not None:
    # 주어진 객체 정보 리스트
    object_list = transformed_objects

    # 각 클래스에 따른 색상 설정 (BGR 형식)
    color_map = {
        'RASPBERRY PICO': (0, 0, 255),   # 빨간색
        'HOLE': (255, 0, 0),             # 파란색
        'BOOTSEL': (0, 165, 255),        # 주황색
        'OSCILLATOR': (0, 255, 0),       # 초록색
        'USB': (0, 255, 255),            # 노란색
        'CHIPSET': (255, 0, 255)         # 보라색
    }

    thickness = 2  # 박스 선의 두께

    #count 
    # 각 객체에 대해 박스 그리기
    for obj in object_list:
        class_name = obj['class'] #객체의 클래스 이름을가져옴
        start_point = tuple(obj['start_point']) #튜플로 변환 하여 start_point와 end_point에 저장
        end_point = tuple(obj['end_point'])
        
        # 클래스에 해당하는 색상 가져오기
        color = color_map.get(class_name, (255, 255, 255))  # 기본값은 흰색
        
        # 박스 그리기
        cv2.rectangle(img, start_point, end_point, color, thickness)
        
        # 클래스 이름을 박스 위에 텍스트로 표시
        cv2.putText(img, class_name, (start_point[0], start_point[1] - 10), #텍스트의 위치는 바운딩 박스의 좌상단 바로 위로 설정합니다.
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # 결과 이미지 표시
    cv2.imshow("Annotated Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("이미지를 불러오지 못했습니다. 경로를 확인하세요.")

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jeonghan/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [9]:
'''
import cv2
# 이미지 불러오기# 이미지 파일 경로 설정
img = cv2.imread(img_path)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()'''


'\nimport cv2\n# 이미지 불러오기# 이미지 파일 경로 설정\nimg = cv2.imread(img_path)\ncv2.imshow("Image", img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()'

In [10]:
response.status_code  # 4 is bad

200